In [1]:
import pylangacq as pla

In [215]:
pla.__version__

'0.11.0'

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
import glob

In [6]:
import os

In [7]:
import pickle

In [8]:
import torch

Data grab

In [334]:
cookie_data = pla.read_chat('./Pitt/*/cookie/*.cha')

In [10]:
sentence_data = pla.read_chat('./Pitt/*/sentence/*.cha')

In [265]:
recall_data = pla.read_chat('./Pitt/*/recall/*.cha')

In [150]:
sentence_fnames = glob.glob(os.getcwd()+'/Pitt/*/sentence/*.cha')

In [151]:
sentence_fnames = sorted(sentence_fnames)

In [335]:
# Control, Dementia
cookie_fnames = glob.glob(os.getcwd()+'/Pitt/*/cookie/*.cha')

In [336]:
cookie_fnames = sorted(cookie_fnames)

In [266]:
recall_fnames = glob.glob(os.getcwd()+'/Pitt/*/recall/*.cha')

In [268]:
recall_fnames = sorted(recall_fnames)

In [88]:
#samples
len(sentence_fnames+cookie_fnames+recall_fnames)

1054

In [89]:
len(cookie_data)

552

In [90]:
len(sentence_data)

240

In [91]:
len(recall_data)

262

In [92]:
# for random batching/data grabbing 
fnames_dict = {i:fname for (i,fname) in enumerate(cookie_fnames+sentence_fnames+recall_fnames)}

In [93]:
# torch.save(fnames_dict,'fnames_cookie_sentence_recall_dict.pt')

Targets (MMSE scores)

In [78]:
# fill missing mmse with group's mean mmse
def get_targets(dataset,filenames):
    
    targets = []
    groups = {}
    
#     for fname in sorted(filenames):
    for fname in filenames:
        # PAR MMSE
#         print(fname)
# MOD groupnames to lower -- select few mistyped 
        target = dataset.headers()[fname]['Participants']['PAR']['education']
        group = dataset.headers()[fname]['Participants']['PAR']['group'].lower()
        
        # to be replaced with group mean
        if target is '':
            targets.append(group)
            
        else:
            targets.append(int(target))
            if group.lower() in groups.keys():
                groups[group.lower()].append(int(target))
            else:
                groups[group.lower()] = [int(target)]
                
#     print('targets',targets)            
#     print('groups',groups)
    
    for g in groups.keys():
        m = np.mean(groups[g])
        groups[g] = int(m) if m % 1 > 5 else int(m) + 1
    
    #TODO replace with tensor
    targetseries = pd.Series(targets)
    
#     print('groups_mean',groups)
    
    for g in groups.keys():
        targetseries[[i for i,x in enumerate(targets) if x==g]] = groups[g]
# Uncomment to return dict of group-averages
#     return groups
    return targetseries

In [130]:
cookie_data_targets = get_targets(cookie_data,cookie_fnames)

In [63]:
get_targets(cookie_data,cookie_fnames)

{'Control': 30,
 'ProbableAD': 19,
 'MCI': 28,
 'Memory': 30,
 'Vascular': 17,
 'PossibleAD': 20,
 'Probable': 19,
 'Other': 24}

In [50]:
# torch.save(cookie_data_targets,'cookie_data_targets.pt')

In [51]:
cookie_target_dict = {i:fname for (i,fname) in enumerate(cookie_data_targets)}

In [52]:
# torch.save(cookie_target_dict,'cookie_target_dict.pt')

In [53]:
sentence_data_targets = get_targets(sentence_data,sentence_fnames)

In [62]:
get_targets(sentence_data,sentence_fnames)

{'ProbableAD': 19,
 'MCI': 28,
 'Memory': 30,
 'Vascular': 17,
 'PossibleAD': 21,
 'Control': 30,
 'Probable': 19,
 'Other': 24}

In [54]:
# torch.save(sentence_data_targets,'sentence_data_targets.pt')

In [55]:
sentence_target_dict = {i+len(cookie_data):fname for (i,fname) in enumerate(sentence_data_targets)}

In [56]:
# torch.save(sentence_target_dict,'sentence_target_dict.pt')

In [79]:
recall_data_targets = get_targets(recall_data,recall_fnames)

In [81]:
torch.save(recall_data_targets,'recall_data_targets.pt')

In [98]:
recall_target_dict = {i+len(sentence_data)+len(cookie_data):fname for (i,fname) in enumerate(recall_data_targets)}

In [262]:
torch.save(recall_target_dict,'recall_target_dict.pt')

In [61]:
get_targets(recall_data,recall_fnames)

{'ProbableAD': 19,
 'MCI': 28,
 'Memory': 30,
 'Vascular': 17,
 'PossibleAD': 21,
 'Probable': 19,
 'Other': 24}

In [21]:
len(sentence_data_targets) == len(sentence_data)

True

In [22]:
len(cookie_data_targets) == len(cookie_data)

True

In [58]:
len(recall_data_targets) == len(recall_data)

True

Renumerate data variables for local data grabbing: vocabset, posset building

In [132]:
cookie_fnames = {i:fname for (i,fname) in enumerate(cookie_fnames)}

In [41]:
torch.save(cookie_fnames,'cookie_fnames_dict.pt')

In [144]:
# for saving -- 
#sentence_fnames = {i+len(cookie_data):fname for (i,fname) in enumerate(sentence_fnames)}

In [46]:
# torch.save(sentence_fnames,'sentence_fnames_dict.pt')

In [152]:
sentence_fnames = {i:fname for (i,fname) in enumerate(sentence_fnames)}

In [269]:
# for saving -- 
#recall_fnames = {i+len(cookie_data)+len(sentence_data):fname for (i,fname) in enumerate(recall_fnames)}

In [271]:
#torch.save(recall_fnames,'recall_fnames_dict.pt')

In [159]:
recall_fnames = {i:fname for (i,fname) in enumerate(recall_fnames)}

Load addb vocab glove word embeddings 

In [100]:
import torch

In [101]:
glove_path = './PretrainedWordEmb/'

In [102]:
addbvocab = pickle.load(open(f'{glove_path}/addb.vocab.glove.42B.300_words.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: './PretrainedWordEmb//addb.vocab.glove.42B.300_words.pkl'

In [38]:
len(addbvocab)

2188

---------------------------------------------

BUILD VOCAB

In [161]:
vocabset = set()

In [162]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data),(recall_fnames,recall_data)]: 
    for i in range(len(corpus)):
        tokens = set([tokenraw for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        vocabset.update(tokens)

In [163]:
len(vocabset)

2839

In [22]:
# len(vocabset)

2188

db Vocab as gloVe emb 

In [43]:
sorted(list(addbvocab)) == sorted(list(vocabset))

True

BUILD MOR: POS + GRAMMATICAL CATEGORIES

In [165]:
posset = set()

In [166]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data),(recall_fnames,recall_data)]: 
    for i in range(len(corpus)):
        pos = set([pos for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        posset.update(pos)

In [167]:
# to concat to embedding tensor 
pos_dict = {pos:i for (i,pos) in enumerate(posset)}

In [168]:
pos_dict

{'': 0,
 'OVER#V': 1,
 '+..?': 2,
 'DIS#N': 3,
 'MID#N': 4,
 'PRO:DEM': 5,
 'PRO:OBJ': 6,
 'NEO': 7,
 'PRO:INT': 8,
 'DET:ART': 9,
 'PREP': 10,
 'N:GERUND': 11,
 'DET:DEM': 12,
 'QN': 13,
 'PRO:REFL': 14,
 'RE#V': 15,
 '+/.': 16,
 'N': 17,
 'DET:NUM': 18,
 '+"/.': 19,
 'GRAND#ADJ': 20,
 'V': 21,
 'COMP': 22,
 'UN#ADJ': 23,
 '.': 24,
 'OVER#N:GERUND': 25,
 'MINI#N': 26,
 'PRO:EXIST': 27,
 'UN#PART': 28,
 'CO': 29,
 'N:PROP': 30,
 'GRAND#N': 31,
 'ON': 32,
 'N:PT': 33,
 'MOD': 34,
 'AUX': 35,
 'COP': 36,
 'N:ADJ': 37,
 'OUT#PART': 38,
 'UN#ADV': 39,
 'PRO:POSS': 40,
 'PRE#V': 41,
 'PRE#PART': 42,
 'META': 43,
 '+//?': 44,
 'UN#N': 45,
 'UP#PART': 46,
 'BEG': 47,
 'INF': 48,
 'OVER#PART': 49,
 'DET:POSS': 50,
 'DIS#V': 51,
 '+//.': 52,
 'PRO:REL': 53,
 'UP#V': 54,
 '+/?': 55,
 'POST': 56,
 '+".': 57,
 'ADV:TEM': 58,
 'CONJ': 59,
 'END': 60,
 'IN#ADJ': 61,
 '!': 62,
 'CM': 63,
 'ADJ': 64,
 'PRO:PER': 65,
 'PRO:INDEF': 66,
 '+.': 67,
 'PRO:SUB': 68,
 'N:LET': 69,
 'NEG': 70,
 '+...': 71,
 '

In [38]:
# torch.save(pos_dict,'pos_dict.pt')

In [169]:
torch.save(pos_dict,'pos_cookie_sent_recall_dict.pt')

In [25]:
# len(posset)

71

In [170]:
len(posset)

76

--------------------------------------------------------------------------------------------------

VOCAB EMB

In [178]:
#https://github.com/Blosc/bcolz
import bcolz

In [31]:
# import zipfile

In [32]:
# with zipfile.ZipFile('./PretrainedWordEmb/glove.42B.300d.zip', 'r') as zip_ref:
#     zip_ref.extractall('./PretrainedWordEmb/')

load pretrained glove (common crawl 42B tokens, 1.9M vocab, uncased, 300d vectors)

In [52]:
# with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         break
    

In [58]:
# words = []
# idx = 0
# word2idx = {}
# vectors = bcolz.carray(np.zeros(1), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')

# with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         vect = np.array(line[1:]).astype(np.float)
#         vectors.append(vect)

In [30]:
# vectors = bcolz.carray(vectors[1:].reshape((-1,300)), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')
# vectors.flush()

In [31]:
# pickle.dump(words, open(f'{glove_path}/glove.42B.300_words.pkl', 'wb'))
# pickle.dump(word2idx, open(f'{glove_path}/glove.42B.300_idx.pkl', 'wb'))

In [188]:
vectors = bcolz.open(f'{glove_path}/glove.42B.300.dat')[:]

In [189]:
words = pickle.load(open(f'{glove_path}/glove.42B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/glove.42B.300_idx.pkl', 'rb'))

In [192]:
'UNK' in words

False

In [193]:
type(words)

list

In [194]:
type(word2idx)

dict

In [195]:
words.append('UNK')

In [196]:
vectors.shape

(1917494, 300)

In [197]:
# UNK token -- average of embeddings
average_vec = np.mean(vectors, axis=0)
# print(average_vec)

In [198]:
vectors = np.concatenate((vectors,np.reshape(average_vec,(1,300))))

In [252]:
# len(vectors)

1917495

In [199]:
len(vectors)

1917495

In [200]:
word2idx['UNK'] = len(vectors) - 1

In [201]:
len(words)

1917495

In [221]:
import nltk

In [283]:
import string
from nltk.tokenize.treebank import TreebankWordTokenizer

def get_glovembs(sample):
    
    def handle_contractions(x):
        tokenizer = TreebankWordTokenizer()
        x = tokenizer.tokenize(x)
        x = ' '.join(x)
        return x
    
    # preprocess acc to pretrained embeddings 
    # lower
    # replace _, -
    # rm contractions
    # rm nonascii chars
    
    sample = [s for s in sample]
    
    preprocessample = [handle_contractions(t).split()[0] for t in [token.lower().replace('_','-') for token in sample]]
    
    
    preprocessample = [''.join([i if ord(i) < 128 else '' for i in w]) for w in preprocessample]
    
    preprocessample = [t.translate(str.maketrans('', '', string.punctuation)) for t in preprocessample]
    
# uncomment for traceunks = get_glovembs(vocabset)
#     return set([word for word in preprocessample if word not in words])
    # {w: vectors[word2idx[w]] for w in words}
    return {sample[i]: torch.tensor(vectors[word2idx[preprocessample[i]]]) if preprocessample[i] in words else torch.tensor(vectors[word2idx['UNK']]) for i in range(len(sample))}

In [322]:
import string
from nltk.tokenize.treebank import TreebankWordTokenizer

def get_glovembs(sample):
    
#     unks = set()
    
    def handle_contractions(x):
        tokenizer = TreebankWordTokenizer()
        x = tokenizer.tokenize(x)
        x = ' '.join(x)
        return x
#     def list_tokenize(document):
#         sentences = nltk.sent_tokenize(document)
#         return [nltk.word_tokenize(sent) for sent in sentences]
    
    # preprocess acc to pretrained embeddings 
    # lower
    # replace _, -
    # rm contractions
    # rm nonascii chars
    
    sample = [s for s in sample]
    
    wordvecs = []
    
#     print('sample',sample)
    preprocessample = [handle_contractions(t).split()[0] for t in [token.lower().replace('_','-') for token in sample]]
#     print('preprocessample',preprocessample)
    
    
    preprocessample = [''.join([i if ord(i) < 128 else '' for i in w]) for w in preprocessample]
    
    for preprocessedtkn in preprocessample:
        wordvecs.extend(preprocessedtkn.split('-'))
    preprocessample = wordvecs
    
        
#     for i in range(len(sample)):
#         if preprocessample[i] not in words:
#             unks.add(preprocessample[i])
#     uncomment to indicate which are 'UNK'
#     return unks
    return {sample[i]: torch.tensor(vectors[word2idx[preprocessample[i]]]) if preprocessample[i] in words else torch.tensor(vectors[word2idx['UNK']]) for i in range(len(sample))}

In [313]:
get_glovembs(vocabset)

{'',
 '..',
 '//',
 'alrightie',
 'cappdf',
 'cookeiejar',
 'hmhunh',
 'hunhunh',
 'nuhhuh',
 'sewickly',
 'thanksiving',
 'unhunh'}

In [310]:
len(all_vecs)

2839

In [245]:
unks = get_glovembs(vocabset)

In [246]:
len(unks)

12

In [247]:
unks

{'',
 '..',
 '//',
 'alrightie',
 'cappdf',
 'cookeiejar',
 'hmhunh',
 'hunhunh',
 'nuhhuh',
 'sewickly',
 'thanksiving',
 'unhunh'}

In [217]:
len(unks)

40

In [304]:
emptylist = []

In [307]:
[emptylist.extend(u.split('-')) for u in prev_unks]

SyntaxError: invalid syntax (<ipython-input-307-bb94483d548a>, line 1)

In [306]:
emptylist

['',
 'oh',
 'god',
 'mister',
 'miller',
 'holy',
 'mackerel',
 'little',
 'red',
 'riding',
 'hood',
 'god',
 'bless',
 'you',
 'mister',
 'martin',
 'unhunh',
 'hmhunh',
 'oh',
 'dear',
 'oh',
 'gosh',
 'council',
 'care',
 'hunhunh',
 'i',
 "don't",
 'know',
 'i',
 'mean',
 '..',
 'how',
 'come',
 'sewickly',
 'dave',
 'branton',
 'joyce',
 'kilmer',
 'what',
 'about',
 'how',
 'about',
 '//',
 'oh',
 'brother',
 'the',
 'window',
 'of',
 'the',
 'xxx',
 'doctor',
 'dick',
 'oh',
 'lord',
 'oh',
 'gee',
 'nuhhuh',
 'oh',
 'boy',
 'thanksiving',
 'tazmania',
 'dutch',
 'a',
 'lot',
 'of',
 'cookeiejar',
 'my',
 'goodness',
 'lots',
 'of',
 'pardon',
 'me',
 'cappdf',
 'alrightie',
 'lee',
 'a']

In [291]:
prev_unks = {'',
 '..',
 '//',
 'a-lot-of',
 'alrightie',
 'cappdf',
 'cookeiejar',
 '-care',
 'dave-branton',
 'doctor-dick',
 'god-bless-you',
 'hmhunh',
 'holy-mackerel',
 'how-about',
 'how-come',
 'hunhunh',
 "i-don't-know",
 'i-mean',
 'joyce-kilmer',
 'lee-a',
 'little-red-riding-hood',
 'lots-of',
 'mister-martin',
 'mister-miller',
 'my-goodness',
 'nuhhuh',
 'oh-boy',
 'oh-brother',
 'oh-dear',
 'oh-gee',
 'oh-god',
 'oh-gosh',
 'oh-lord',
 'pardon-me',
 'sewickly',
 'tazmania-dutch',
 'thanksiving',
 'the-window-of-the-xxx',
 'unhunh',
 'what-about'}

In [294]:
emptylist = []

In [296]:
[u.split('-') for u in prev_unks]

[[''],
 ['oh', 'god'],
 ['mister', 'miller'],
 ['holy', 'mackerel'],
 ['little', 'red', 'riding', 'hood'],
 ['god', 'bless', 'you'],
 ['mister', 'martin'],
 ['unhunh'],
 ['hmhunh'],
 ['oh', 'dear'],
 ['oh', 'gosh'],
 ['council', 'care'],
 ['hunhunh'],
 ['i', "don't", 'know'],
 ['i', 'mean'],
 ['..'],
 ['how', 'come'],
 ['sewickly'],
 ['dave', 'branton'],
 ['joyce', 'kilmer'],
 ['what', 'about'],
 ['how', 'about'],
 ['//'],
 ['oh', 'brother'],
 ['the', 'window', 'of', 'the', 'xxx'],
 ['doctor', 'dick'],
 ['oh', 'lord'],
 ['oh', 'gee'],
 ['nuhhuh'],
 ['oh', 'boy'],
 ['thanksiving'],
 ['tazmania', 'dutch'],
 ['a', 'lot', 'of'],
 ['cookeiejar'],
 ['my', 'goodness'],
 ['lots', 'of'],
 ['pardon', 'me'],
 ['cappdf'],
 ['alrightie'],
 ['lee', 'a']]

In [214]:
unks

{'',
 '..',
 '//',
 'a-lot-of',
 'alrightie',
 'cappdf',
 'cookeiejar',
 'council-care',
 'dave-branton',
 'doctor-dick',
 'god-bless-you',
 'hmhunh',
 'holy-mackerel',
 'how-about',
 'how-come',
 'hunhunh',
 "i-don't-know",
 'i-mean',
 'joyce-kilmer',
 'lee-a',
 'little-red-riding-hood',
 'lots-of',
 'mister-martin',
 'mister-miller',
 'my-goodness',
 'nuhhuh',
 'oh-boy',
 'oh-brother',
 'oh-dear',
 'oh-gee',
 'oh-god',
 'oh-gosh',
 'oh-lord',
 'pardon-me',
 'sewickly',
 'tazmania-dutch',
 'thanksiving',
 'the-window-of-the-xxx',
 'unhunh',
 'what-about'}

In [323]:
addb_vocab_embs = get_glovembs(vocabset)

In [257]:
# len(addb_vocab_embs)

2188

In [324]:
len(addb_vocab_embs)

2839

In [252]:
len(addb_vocab_embs)

2839

In [325]:
addb_vocab_embs['UNK']

KeyError: 'UNK'

In [327]:
addb_vocab_embs['I']

tensor([-6.0053e-02, -1.0321e-01, -1.6738e-01,  1.9922e-01,  5.9712e-02,
         3.1170e-02, -4.4661e+00,  1.3585e-01,  2.7248e-01, -5.9720e-01,
         2.3723e-01,  2.0766e-02, -2.6883e-01, -2.4141e-01, -1.8461e-01,
        -6.4325e-02, -1.6662e-01, -2.2890e-01, -2.0721e-01, -3.6173e-02,
         4.4040e-01, -6.7288e-02,  1.1337e-01,  3.3058e-01, -9.1099e-02,
         5.3504e-03, -3.1603e-01, -8.9506e-02, -2.1208e-01,  8.6033e-02,
        -2.3813e-02, -1.7691e-01,  4.6761e-01,  8.7031e-02,  1.6213e-01,
         1.2928e-02, -3.2798e-01, -1.4660e-01,  7.0218e-02,  1.3093e-01,
        -1.2166e-01, -1.9080e-01,  1.9738e-01, -3.8298e-01,  3.2726e-01,
         3.2674e-01,  1.3125e-01, -2.8537e-01, -1.4433e-01, -7.1196e-01,
        -5.6059e-02, -5.5130e-01, -1.1745e-01, -2.5487e-01,  1.5711e-01,
        -3.5474e-02,  2.8145e-01,  4.1805e-02,  2.2557e-02,  1.4275e-01,
         1.6868e-01,  7.0439e-02,  2.1714e-01, -1.0379e-01, -1.9166e-01,
         8.6866e-02,  2.3581e-01, -1.0142e-01, -1.8

In [315]:
vectors[word2idx['UNK']]

array([-6.77110762e-03, -2.38171602e-03,  2.48453616e-02,  3.84871991e-02,
       -1.27718938e-01,  3.31646344e-02,  4.86656168e-01, -1.60207891e-01,
        1.68529167e-02,  1.63224430e-01, -1.02243697e-01, -5.26176498e-02,
        3.46847772e-02, -1.90403945e-02,  3.26858599e-02,  3.54991079e-02,
        6.39390293e-02,  3.84283555e-02, -1.16503242e-02, -2.53978575e-03,
       -1.54851111e-02, -2.30899176e-02,  1.10525923e-02, -2.86016257e-02,
       -2.39241718e-02,  5.43962499e-02, -2.72564042e-03,  2.93424213e-02,
        3.39359506e-02,  5.73053679e-02,  5.95071778e-02,  1.09199854e-04,
       -2.39850050e-02,  2.59529588e-02,  1.05694563e-02,  7.69458290e-03,
        1.86652275e-02,  1.77983629e-02, -2.53548389e-02,  2.43759036e-02,
        2.54368294e-02, -5.54019334e-02,  2.59849416e-02,  4.13441975e-02,
        1.87303416e-02,  3.68259799e-03, -3.21201589e-02,  5.68750757e-02,
        4.68697892e-02, -1.33935990e-02, -1.44603463e-03,  4.74672429e-04,
        2.68379948e-02,  

In [328]:
addb_vocab_embs['CLITIC']

tensor([ 7.7695e-02,  4.1614e-01, -1.7101e-01,  1.7692e-01, -2.7406e-02,
         5.4102e-01, -3.6129e+00,  1.6151e-01,  1.9711e-01,  2.8919e-02,
         2.8745e-01, -9.3931e-02,  2.1711e-01, -3.6525e-01,  2.0094e-01,
         4.0357e-01,  1.1149e-01, -1.9783e-02, -6.8662e-02,  4.0455e-02,
        -1.0114e-01,  6.2720e-02, -2.4347e-01, -2.0251e-01, -4.5643e-01,
        -1.6537e-01, -1.4941e-02, -7.6818e-02, -1.8922e-02, -3.7106e-01,
         1.6106e-01, -3.7111e-01,  3.8205e-01,  2.5370e-01, -3.9167e-01,
        -3.6314e-03, -3.0197e-01, -1.9817e-01, -2.1655e-01,  1.2998e-01,
        -1.1480e-01,  3.0046e-01, -7.9314e-01,  1.7068e-01,  2.0446e-01,
        -4.5931e-02, -1.9753e-01,  3.2807e-01,  5.5548e-01, -4.4488e-01,
        -8.4063e-01, -3.4806e-02, -1.2154e-01, -3.2563e-01,  4.6772e-02,
        -6.2888e-02,  5.5237e-01, -8.4866e-02, -9.6028e-02, -1.1892e-01,
         3.1771e-01, -5.8550e-02,  3.1959e-01, -4.6279e-01,  5.1349e-01,
        -8.2146e-02, -1.3512e-01,  5.3531e-02, -3.8

In [329]:
torch.from_numpy(vectors[word2idx['UNK']])

tensor([-6.7711e-03, -2.3817e-03,  2.4845e-02,  3.8487e-02, -1.2772e-01,
         3.3165e-02,  4.8666e-01, -1.6021e-01,  1.6853e-02,  1.6322e-01,
        -1.0224e-01, -5.2618e-02,  3.4685e-02, -1.9040e-02,  3.2686e-02,
         3.5499e-02,  6.3939e-02,  3.8428e-02, -1.1650e-02, -2.5398e-03,
        -1.5485e-02, -2.3090e-02,  1.1053e-02, -2.8602e-02, -2.3924e-02,
         5.4396e-02, -2.7256e-03,  2.9342e-02,  3.3936e-02,  5.7305e-02,
         5.9507e-02,  1.0920e-04, -2.3985e-02,  2.5953e-02,  1.0569e-02,
         7.6946e-03,  1.8665e-02,  1.7798e-02, -2.5355e-02,  2.4376e-02,
         2.5437e-02, -5.5402e-02,  2.5985e-02,  4.1344e-02,  1.8730e-02,
         3.6826e-03, -3.2120e-02,  5.6875e-02,  4.6870e-02, -1.3394e-02,
        -1.4460e-03,  4.7467e-04,  2.6838e-02,  8.0426e-06, -2.1895e-02,
         1.8322e-02, -3.3935e-02, -2.6631e-02,  2.2321e-02, -1.9475e-05,
         7.3048e-03,  2.9189e-02, -2.0734e-02, -1.1434e-02,  1.0040e-02,
        -2.9709e-02, -6.2757e-03, -4.6265e-02, -4.0

In [330]:
# replace CLITIC emb with UNK token (mistaken for word 'clitic')
# mistakenly saved as an np array...overwrote this key with torch.from_numpy(vocab_emb['CLITIC']): saved as addb.vocab.emb.glove.42B.300
addb_vocab_embs['CLITIC'] = torch.from_numpy(vectors[word2idx['UNK']])

In [331]:
pickle.dump(addb_vocab_embs, open(f'{glove_path}/addb.csr.vocab.glove.42B.300dwords.pkl', 'wb'))

In [332]:
addbvocab = pickle.load(open(f'{glove_path}/addb.csr.vocab.glove.42B.300dwords.pkl', 'rb'))

In [282]:
# len(addbvocab)

2188

In [333]:
len(addbvocab)

2839

In [258]:
list(addbvocab) == list(addb_vocab_embs)

True

In [152]:
len(vocabset_proc)

2188

In [154]:
# list generated from preprocessing steps subsumed into get_glovembs()
len(unks)

26

In [ ]:
# ignore CLITIC

In [183]:
unks

['cookeiejar',
 "i-don't-know",
 'cappdf',
 '//',
 '',
 '//',
 'alrightie',
 '..',
 'oh-boy',
 'hmhunh',
 'sewickly',
 'tazmania-dutch',
 'doctor-dick',
 'how-about',
 'god-bless-you',
 '',
 'oh-dear',
 'how-come',
 'i-mean',
 'what-about',
 'joyce-kilmer',
 'the-window-of-the-xxx',
 'dave-branton',
 'lots-of',
 'a-lot-of',
 'lee-a']